# Example 5-1: Finding the Sun Position Vector
### _Fundamentals of Astrodynamics and Applications_, 5th Ed., 2022, pp. 286-287

This notebook demonstrates calculating the geocentric position of the Sun (mean-of-date) given a date and time.

## Install and Import Libraries
---

First, install `valladopy` if it doesn't already exist in your environment:

In [1]:
!pip install valladopy==0.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 3.2 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: valladopy
    Found existing installation: valladopy 0.3.0
    Uninstalling valladopy-0.3.0:
      Successfully uninstalled valladopy-0.3.0


Import `numpy` and the relevant `valladopy` modules:

In [2]:
import numpy as np
from valladopy.astro.celestial import sun
from valladopy.mathtime.julian_date import jday

## Problem Definition
---

GIVEN: &ensp;April 1, 2006, 23:58:54.816 UTC<br>
FIND: &emsp;Geocentric position vector of the Sun (MOD)

In [3]:
year = 2006
month = 4
day = 1
hour = 23
minute = 58
second = 54.816

## Solution
---

We begin by finding the Julian Date using **Algorithm 14**:

$$
JD = 367(yr)
- \text{INT}\left\{ \frac{7\left\{ yr + \text{INT} \left( \frac{mo + 9}{12} \right) \right\}}{4} \right\}
+ \text{INT}\left( \frac{275\,mo}{9} \right)
+ d + 1{,}721{,}013.5
+ \frac{h\,3600 + min\,60 + s}{86400}
$$

Which is done with the `jday` routine:

In [4]:
jd, jdfrac = jday(year, month, day, hour, minute, second)

print(f'Julian Date:\t{jd + jdfrac:.4f}')

Julian Date:	2453827.4992


**Algorithm 29** outlines the steps to find the Sun position vector. First find the Julian centuries of UT1:

$$
T_{UT1} = \frac{JD_{UT1} - 2,451,545.0}{36,525}
$$

The mean ecliptic longitude of the Sun is:

$$
\bar{\lambda}_{ecliptic \ \odot} = 280.460^{\circ} + 36,000.771285 \ T_{UT1}
$$

And if we let $T_{TDB} \cong T_{UT1}$, then the mean anomaly is:

$$
M_{\odot} = 357.528^{\circ} + 35,999.050957 \ T_{UT1}
$$

The ecliptic longitude of the Sun is:

$$
\lambda_{ecliptic \ \odot} = \bar{\lambda}_{ecliptic \ \odot} + 1.915^{\circ} \sin(M_{\odot}) + 0.020 \sin(2 M_{\odot})
$$

The position magnitude of the Sun is:

$$
r_{\odot} = 1.00014 - 0.01671 \cos(M_{\odot}) - 0.00014 \cos(2 M_{\odot})
$$

The obliquity of the ecliptic can be approximated as:

$$
\epsilon = 23.439291^{\circ} - 0.01461 \ T_{TDB} \ \cong \ 23.439291^{\circ} - 0.01461 \ T_{UT1}
$$

Finally, the position vector in geocentric equatorial coordinates (MOD) is:

$$
\vec{r_{\odot}} = r_{\odot} \begin{bmatrix}
\cos(\lambda_{ecliptic \ \odot}) \\
\cos(\epsilon) \sin(\lambda_{ecliptic \ \odot}) \\
\sin(\epsilon) \sin(\lambda_{ecliptic \ \odot})
\end{bmatrix} \
\text{AU}
$$

Call the `sun.position` routine to accomplish this:

In [5]:
rsun, rtasc, decl = sun.position(jd + jdfrac)

print(f'Sun position vector:\t{rsun}\tkm')

Sun position vector:	[1.46186375e+08 2.87882000e+07 1.24807271e+07]	km


Note that the output for the sun vector is in kilometers, i.e., it is converted within the function.

The routine above also finds the right ascension and declination, which use the simplified formulas:

$$
\begin{aligned}
\tan(\alpha_{\odot}) &= \cos(\epsilon) \tan(\lambda_{ecliptic \ \odot}) \\
\\
\sin(\delta_{\odot}) &= \sin(\epsilon) \sin(\lambda_{ecliptic \ \odot})
\end{aligned}
$$

In [6]:
print(f'Right ascension:\t{np.degrees(rtasc):.6}\tdeg')
print(f'Declination:\t\t{np.degrees(decl):.6}\tdeg')

Right ascension:	11.1406	deg
Declination:		4.7883	deg
